In [2]:
import pandas as pd
import pdb
import numpy as np
import glob

In [ ]:
import zipfile
import os
import glob
import pandas as pd
import numpy as np
with zipfile.ZipFile('training_setB.zip', 'r') as zip_ref:
    zip_ref.extractall('training_setB')
files = glob.glob('training_setB/training_setB/*.psv')

df_list = []
for ind, f in enumerate(files):
    # print(f)
    # print(f.split('/')[1])
    # print(f.split('/')[2])
    # print(f.split('/')[2].split('.')[0])
    patient_id = f.split('/')[2].split('.')[0]
    # patient_id = f.split('/')[1].split('.')[0]

    df = pd.read_csv(f, sep='|')
    df = df.assign(patient=patient_id)
    df.loc[df[df['SepsisLabel'] == 1].head(6).index.values, 'SepsisLabel'] = 1

    if ind % 1000 == 0:
        print(ind)
    df_list.append(df)

    # break

# Save in pickle file
df = pd.concat(df_list)
df = df.reset_index(drop=True)
df.to_pickle('combined.pkl')


In [3]:
import pandas as pd
import numpy as np
import pdb
import os
import shutil

In [4]:
try:
    if os.path.exists('feats'):
        shutil.rmtree('feats')
    os.makedirs('feats')
except Exception as e:
    print(e)

In [5]:
df = pd.read_pickle('combined.pkl')

In [ ]:
import pickle
file_path = 'combined.pkl'
with open(file_path, 'rb') as file:
    data = pickle.load(file)
print(data)

In [ ]:
print('Percentage Missing:')
print(df.isna().sum()/len(df))
cols_to_drop = ['Unit2', 'ICULOS']
df = df.drop(cols_to_drop, axis=1)
cols_cont = ['HR', 'MAP', 'O2Sat', 'SBP', 'Resp']
cols_to_bin = ['Unit1', 'Gender', 'HospAdmTime', 'Age', 'DBP', 'Temp', 'Glucose', 'Potassium', 'Hct', 'FiO2', 'Hgb', 'pH', 'BUN', 'WBC', 'Magnesium', 'Creatinine', 'Platelets', 'Calcium', 'PaCO2', 'BaseExcess', 'Chloride', 'HCO3', 'Phosphate', 'EtCO2', 'SaO2', 'PTT', 'Lactate', 'AST', 'Alkalinephos', 'Bilirubin_total', 'TroponinI', 'Fibrinogen', 'Bilirubin_direct']

In [8]:
patients_training_data = df['patient'].unique()
np.random.shuffle(patients_training_data)
patients_training_data = patients_training_data[0:-6000]

df_mean_std = df[df['patient'].isin(patients_training_data)].describe().loc[['mean', 'std']]
df_mean_std.to_pickle('mean_std_scaling.pkl')

In [ ]:
print('Number of positive training examples:')
sum(df[df['patient'].isin(patients_training_data)]['SepsisLabel']==1)

In [ ]:
#loop through each patient
save_count = 0
windowed_df_list = []
grouped_by_patient = df.groupby('patient')
for patient, group in grouped_by_patient:
    #print(patient)
    group = group.reset_index(drop=True)

    #backfill any missing values
    group = group.assign(HR=group['HR'].fillna(method='bfill').fillna(method='ffill'))
    group = group.assign(MAP=group['MAP'].fillna(method='bfill').fillna(method='ffill'))
    group = group.assign(O2Sat=group['O2Sat'].fillna(method='bfill').fillna(method='ffill'))
    group = group.assign(SBP=group['SBP'].fillna(method='bfill').fillna(method='ffill'))
    group = group.assign(Resp=group['Resp'].fillna(method='bfill').fillna(method='ffill'))

    group = group.assign(HR=(group['HR']-df_mean_std['HR']['mean'])/(df_mean_std['HR']['std']))
    group = group.assign(MAP=(group['MAP']-df_mean_std['MAP']['mean'])/(df_mean_std['MAP']['std']))
    group = group.assign(O2Sat=(group['O2Sat']-df_mean_std['O2Sat']['mean'])/(df_mean_std['O2Sat']['std']))
    group = group.assign(SBP=(group['SBP']-df_mean_std['SBP']['mean'])/(df_mean_std['SBP']['std']))
    group = group.assign(Resp=(group['Resp']-df_mean_std['Resp']['mean'])/(df_mean_std['Resp']['std']))

    #generate windows of 10 hours
    windowed_data = []
    N = len(group)
    win_len = 10
    pred_len = 1
    i = 0
    while(i+win_len+pred_len <= N):
        tmp_data = group.iloc[i:i+win_len]
        tmp_label = group.iloc[i+win_len:i+win_len+pred_len]
        tmp_label = int(any(tmp_label['SepsisLabel']))
        tmp_patient = patient
        i = i+1
        X_cont = tmp_data[cols_cont]
        X_cont = X_cont.values
        if np.isnan(X_cont).any(): continue
        X_binned_dict = {}
        for col_to_bin in cols_to_bin:
            tmp_val = tmp_data[col_to_bin].median()
            if col_to_bin not in ['Gender', 'Unit1']:
                tmp_val = (tmp_val-df_mean_std[col_to_bin]['mean'])/df_mean_std[col_to_bin]['std']

            X_binned_dict[col_to_bin] = tmp_val
        tmp_dict = X_binned_dict
        tmp_dict['X_cont'] = X_cont
        tmp_dict['label'] = tmp_label
        tmp_dict['patient'] = tmp_patient
        windowed_data.append(tmp_dict)
    windowed_data_df = pd.DataFrame(windowed_data)
    windowed_df_list.append(windowed_data_df)D
    patient_id_numeric = ''.join(filter(str.isdigit, patient))
    if patient_id_numeric:
        patient_id_int = int(patient_id_numeric[-5:]) 
    else:
        patient_id_int = 0

    if (patient_id_int % 500) == 0:
        print('patient %i' % patient_id_int)
        windowed_df = pd.concat(windowed_df_list).reset_index(drop=True)
        train = windowed_df[windowed_df['patient'].isin(patients_training_data)].drop('patient', axis=1)
        test = windowed_df[~windowed_df['patient'].isin(patients_training_data)].drop('patient', axis=1)

        train.to_pickle('feats/train_%i.pkl' % save_count)
        test.to_pickle('feats/test_%i.pkl' % save_count)

        windowed_df_list = []
        save_count = save_count + 1

if len(windowed_df_list) > 0:
    windowed_df = pd.concat(windowed_df_list).reset_index(drop=True)
    train = windowed_df[windowed_df['patient'].isin(patients_training_data)].drop('patient', axis=1)
    test = windowed_df[~windowed_df['patient'].isin(patients_training_data)].drop('patient', axis=1)

    train.to_pickle('feats/train_%i.pkl' % save_count)
    test.to_pickle('feats/test_%i.pkl' % save_count)

In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import pdb
import glob
import matplotlib.pyplot as plt
import os
import shutil

In [2]:
try:
    if os.path.exists('plots'):
        shutil.rmtree('plots')
    os.makedirs('plots')
except Exception as e:
    print(e)

In [3]:
train_files = glob.glob('feats/train_*.pkl')
train = []
for train_file in train_files:
    train.append(pd.read_pickle(train_file))

train = pd.concat(train)
test_files = glob.glob('feats/test_*.pkl')
test = []
for test_file in test_files:
    test.append(pd.read_pickle(test_file))

test = pd.concat(test)

In [ ]:
print(train.dtypes)

In [ ]:
numeric_train = train.select_dtypes(include=[np.number])
print(numeric_train)

In [ ]:
corr = numeric_train.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
plt.figure(figsize=(16,9))
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.tight_layout()
plt.savefig('plots/heatmap.png', dpi=250)
plt.show()
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

### Train the Model

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.regularizers import l2
from keras.layers import TimeDistributed, Bidirectional, BatchNormalization, Dropout, Input, Add, Masking
from tensorflow.keras.layers import LayerNormalization
from keras import Model
import pdb
import pandas as pd
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
import glob
import sys
from sklearn.metrics import auc
import pickle

In [5]:
BATCH_SIZE = 64
INPUT_SEQ_LEN_MODEL1 = 10
INPUT_NUM_CH_MODEL1 = 5
INPUT_FEATS_MODEL2 = 33

In [6]:
def roc(predictions, true, filename):
    predictions = predictions.flatten()
    true = true.flatten()

    thresh_vals = np.linspace(np.min(predictions), np.max(predictions), 50)
    results = []
    for thresh in thresh_vals:
        tmp_predictions = (predictions > thresh).astype(int)
        tn, fp, fn, tp = confusion_matrix(true, tmp_predictions).ravel()
        tpr = tp/(tp+fn)
        fpr = fp/(tn+fp)
        acc = (tp+tn)/(tn+fp+fn+tp)

        tmp_dict = {'acc': acc, 'tpr': tpr, 'fpr': fpr, 'thresh': thresh}
        results.append(tmp_dict)

    results = pd.DataFrame(results)
    results = results.sort_values(by='thresh', ascending=False)

    #calculate the AUC
    AUC = auc(results['fpr'].values, results['tpr'].values)

    plt.figure(figsize=(10,6))
    plt.plot(results['fpr'], results['tpr'], '*-')
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC\nAUC=%.2f' % AUC)
    plt.tight_layout()
    plt.savefig('plots/ROC_%s.png' % filename, dpi=250)
    plt.show()

    results = results.sort_values(by='acc', ascending=False)
    final_thresh = results.head(1)['thresh'].values[0]

    return results, final_thresh

In [7]:
train = pd.read_pickle('train.pkl').reset_index(drop=True)
N = len(train)
train = train.sample(frac=1).reset_index(drop=True)
val = train[:int(N*0.2)]
train = train[int(N*0.2):]

y_train = np.asarray(list(train['label']))
y_train = to_categorical(y_train)
X_train_cont = np.asarray(list(train['X_cont'].values))
X_train_cat = train.drop(['X_cont', 'label'], axis=1)
X_train_cat[X_train_cat.isna()] = np.pi
X_train_cat = np.asarray(list(X_train_cat.values))

y_val = np.asarray(list(val['label']))
y_val = to_categorical(y_val)
X_val_cont = np.asarray(list(val['X_cont'].values))
X_val_cat = val.drop(['X_cont', 'label'], axis=1)

#replace NaN with pi
X_val_cat[X_val_cat.isna()] = np.pi
X_val_cat = np.asarray(list(X_val_cat.values))

#test data
test = pd.read_pickle('test.pkl')
y_test = np.asarray(list(test['label']))
y_test = to_categorical(y_test)
X_test_cont = np.asarray(list(test['X_cont'].values))
X_test_cat = test.drop(['X_cont', 'label'], axis=1)

X_test_cat[X_test_cat.isna()] = np.pi
X_test_cat = np.asarray(list(X_test_cat.values))

In [ ]:
count_class_0 = np.sum(y_train[:,1].astype(int)==0)
count_class_1 = np.sum(y_train[:, 1].astype(int) == 1)
max_class_counts = np.max((count_class_0, count_class_1))
class_weights = {0: max_class_counts/count_class_0, 1: max_class_counts/count_class_1}
print('class weights -- no sepsis: {}, sepsis: {}'.format(class_weights[0], class_weights[1]))

In [ ]:
input1 = Input(shape=(INPUT_SEQ_LEN_MODEL1, INPUT_NUM_CH_MODEL1))
model1 = Bidirectional(LSTM(100, kernel_regularizer=l2(0.001), return_sequences=True))(input1)
model1 = Bidirectional(LSTM(75, kernel_regularizer=l2(0.001)))(model1)
model1 = Dense(35, kernel_regularizer=l2(0.001), activation='relu')(model1)
model1 = LayerNormalization()(model1)
model1 = Dense(15, kernel_regularizer=l2(0.001), activation='relu')(model1)
model1 = LayerNormalization()(model1)

input2 = Input(shape=(INPUT_FEATS_MODEL2,))
model2 = Masking(mask_value=np.pi)(input2)
model2 = Dense(30, kernel_regularizer=l2(0.001), activation='relu')(model2)
model2 = LayerNormalization()(model2)
model2 = Dense(15, kernel_regularizer=l2(0.001), activation='relu')(model2)
model2 = LayerNormalization()(model2)

model_add = Add()([model1, model2])
output = Dense(2, kernel_regularizer=l2(0.001), activation='softmax')(model_add)

model = Model(inputs=[input1, input2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
checkpoint = ModelCheckpoint('model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4)
history = model.fit([X_train_cont, X_train_cat],
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=50,
                    validation_data=([X_val_cont, X_val_cat], y_val),
                    callbacks=[earlystop, checkpoint],
                    class_weight=class_weights,
                    verbose=1)

#save the history
pickle.dump(history, open('history.pkl', 'wb'))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(history.history['loss'], '*-')
plt.plot(history.history['val_loss'], '*-')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.savefig('plots/loss.png', dpi=250)
plt.show()
plt.close()

In [ ]:
predictions = model.predict([X_val_cont, X_val_cat])

results_df, thresh_final = roc(predictions[:,1].flatten(), y_val[:,1].flatten(), 'val')
results_df = results_df.sort_values(by='fpr')

In [ ]:
predictions = model.predict([X_test_cont, X_test_cat])

results_df, thresh_final = roc(predictions[:,1].flatten(), y_test[:,1].flatten(), 'test')
results_df = results_df.sort_values(by='fpr')

### Generate performance statistics without Epic seismometer

In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
predictions_val = model.predict([X_val_cont, X_val_cat])
y_pred_val = (predictions_val[:, 1] > thresh_final).astype(int)  
y_true_val = y_val[:, 1].astype(int)

predictions_test = model.predict([X_test_cont, X_test_cat])
y_pred_test = (predictions_test[:, 1] > thresh_final).astype(int)
y_true_test = y_test[:, 1].astype(int)

precision_val = precision_score(y_true_val, y_pred_val)
recall_val = recall_score(y_true_val, y_pred_val)
f1_val = f1_score(y_true_val, y_pred_val)
classification_report_val = classification_report(y_true_val, y_pred_val)

precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = f1_score(y_true_test, y_pred_test)
classification_report_test = classification_report(y_true_test, y_pred_test)

print("Validation Set Performance:")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"F1 Score: {f1_val:.4f}")
print("Classification Report:")
print(classification_report_val)

print("\nTest Set Performance:")
print(f"Precision: {precision_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print("Classification Report:")
print(classification_report_test)

with open('classification_report_val.txt', 'w') as val_report:
    val_report.write("Validation Set Classification Report\n")
    val_report.write(classification_report_val)

with open('classification_report_test.txt', 'w') as test_report:
    test_report.write("Test Set Classification Report\n")
    test_report.write(classification_report_test)


### Generate performance statistics with Epic seismometer

In [ ]:
import os
import subprocess
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score

if not os.path.exists("seismometer"):
    print("Cloning Seismometer repository...")
    subprocess.run(["git", "clone", "https://github.com/epic-open-source/seismometer.git"])

print("Installing required dependencies...")
subprocess.run(["pip", "install", "-r", "seismometer/requirements.txt"])

print("Generating predictions...")
test_file = "test.pkl" 
if not os.path.exists(test_file):
    raise FileNotFoundError(f"{test_file} not found. Ensure your preprocessing steps are correct.")

test_data = pd.read_pickle(test_file)
X_test = test_data['X_cont']
y_test = test_data['label']

from keras.models import load_model

model = load_model("model.keras")  
predictions = model.predict([pd.DataFrame(X_test.values.tolist())])
y_pred = (predictions[:, 1] > 0.5).astype(int)  

output_file = "seismometer/predictions.csv"
predictions_df = pd.DataFrame({
    "PatientID": test_data['patient'], 
    "Time": range(len(test_data)),  
    "SepsisLabel": y_pred
})
predictions_df.to_csv(output_file, index=False)
print(f"Predictions saved to {output_file}")

print("Running Seismometer evaluation...")
labels_file = "seismometer/labels.csv" 

labels_df = pd.DataFrame({
    "PatientID": test_data['patient'],  
    "Time": range(len(test_data)),  
    "SepsisLabel": y_test
})
labels_df.to_csv(labels_file, index=False)
print(f"Labels saved to {labels_file}")

evaluator_script = "seismometer/evaluator.py"
subprocess.run(["python", evaluator_script, "--predictions", output_file, "--labels", labels_file])

print("Calculating additional metrics...")
print(classification_report(y_test, y_pred))
roc_auc = roc_auc_score(y_test, predictions[:, 1])
print(f"ROC-AUC Score: {roc_auc:.4f}")